In [1]:
import sys
sys.path.append("../deepnade/buml")
sys.path.append("../deepnade")

import os
from optparse import OptionParser
import Instrumentation
import Backends
import Optimization
import TrainingController
import numpy as np
import Utils
import Data.utils
import scipy.stats
import gc
from Utils.DropoutMask import create_dropout_masks
from Utils.theano_helpers import floatX

import h5py

import NADE

from orderlessNADE import *

In [2]:
parser = get_parser()

os.environ["RESULTSPATH"] = "/Users/marina/Documents/PhD/research/astro_research/code/NADE/output/"
os.environ["DATASETSPATH"] = "/Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/"


In [8]:
options, args =parser.parse_args(["--theano", 
                                  "--form", "MoG", 
                                  "--dataset", "red_wine.hdf5", 
                                  "--training_route", "/folds/1/training/(1|2|3|4|5|6|7|8)",
                                 "--validation_route", "/folds/1/training/9",
                                 "--test_route", "/folds/1/tests/.*",
                                 "--samples_name", "data",
                                 "--hlayers", "2",
                                  "--layerwise",
                                  "--lr", "0.02",
                                  "--wd", "0.02",
                                  "--n_components", "10",
                                  "--epoch_size", "100",
                                  "--momentum", "0.9",
                                  "--units", "100",
                                  "--pretraining_epochs", "5",
                                  "--validation_loops", "20",
                                  "--epochs", "20",
                                  "--normalize",
                                  "--batch_size", "100",
                                  "--show_training_stop", "red_wine"])



In [9]:
if options.theano:
    import NADE
else:
    import npNADE as NADE
    raise Exception("Not implemented yet")

results_route = os.path.join(os.environ["RESULTSPATH"], args[0])
try:
    os.makedirs(results_route)
except OSError:
    pass

console = Backends.Console()
textfile_log = Backends.TextFile(os.path.join(results_route, "NADE_training.log"))
hdf5_backend = Backends.HDF5(results_route, "NADE")
hdf5_backend.write([], "options", options)
hdf5_backend.write([], "svn_revision", Utils.svn.svnversion())
hdf5_backend.write([], "svn_status", Utils.svn.svnstatus())
hdf5_backend.write([], "svn_diff", Utils.svn.svndiff())

In [10]:
training_dataset, validation_dataset, test_dataset, masks_dataset = prep_datasets(options, hdf5_backend)



 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/training/(1|2|3|4|5|6|7|8)
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'training', '(1|2|3|4|5|6|7|8)']
FINAL entries length: 8


 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/training/9
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'training', '9']
FINAL entries length: 1


 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/tests/.*
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'tests', '.*']
FINAL entries length: 1


 BIG DATASET Working on /var/folders/wv/_wh4dxyj3ng73fr13vk_n3700000gn/T/tmp7z2H2p
Entries /folds/1/training/(1|2|3|4|5|6|7|8)
New entries [<HDF5 group "/" (1 members)>]
Pats ['folds', '1', 'training', '(1|2|3|4|5|6|7|8)']
FINAL entries leng

../deepnade/buml/Data/BigDataset.py:103: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return np.array(self.files[element][index].value)  # , order='C')
